# Grid Data Retrieval Testing

The purpose of this notebook is to test the outputs of the grid data retrieval module to ensure data is in the correct format, and retrieved as requested.

### Importing Libraries

In [42]:
import os
from pathlib import Path
import zipfile
import pyarrow as pa
import pyarrow.parquet as pq
import magic
import polars as pl
import pandas as pd


# import pygrib as pg

### Helper Functions

In [43]:
def show_dir_contents(path='.'):
    """List contents of current directory"""
    base = Path(path).resolve()
    print("Contents of current directory:")
    for p in base.rglob('*'):
        print("  -> " + str(p.relative_to(base)))

In [44]:
def print_directory_tree(start_path='.', show_files=False):
    """Print directory tree similar to 'tree' command"""
    for root, dirs, files in os.walk(start_path):
        # Calculate indentation
        level = root.replace(start_path, '').count(os.sep)
        indent = '│   ' * level

        if root == start_path:
            print(f"📁 {os.path.basename(root) or '.'}")
        else:
            print(f"{indent}└── 📁 {os.path.basename(root)}")

        subindent = '│   ' * (level + 1)

        if show_files:
            for f in sorted(files):
                print(f"{subindent}└── 📄 {f}")

In [45]:
def get_mime_type(path):
    """Get MIME type of file using python-magic"""
    m = magic.Magic(mime=True)
    return m.from_file(path)


In [46]:
def read_magic(path, n=8):
    """Read first n bytes of file to determine MIME type"""
    with open(path, 'rb') as f:
        return f.read(n)

In [47]:
def test_parquet_file(filepath, verbose=True, sample_rows=5):
    """
    Test and analyze a Parquet file.

    Args:
        filepath: Path to the Parquet file
        verbose: Print detailed information
        sample_rows: Number of sample rows to display

    Returns:
        Dictionary with test results or None if validation fails
    """
    path = Path(filepath)

    if verbose:
        print(f"\n{'='*60}")
        print(f"🧪 TESTING PARQUET FILE: {path.name}")
        print(f"{'='*60}")

    # Test 0: File exists
    if not path.exists():
        print(f"❌ ERROR: File does not exist")
        return None

    if verbose:
        print(f"📁 Location: {path.parent}")
        print(f"📊 Size: {path.stat().st_size:,} bytes ({path.stat().st_size/1024/1024:.2f} MB)")

    # Test 1: Validate it's a Parquet file
    try:
        # Read metadata first (fastest check)
        metadata = pq.read_metadata(filepath)
        is_parquet = True
        if verbose:
            print(f"✅ VALIDATION: Confirmed as Parquet file")
            print(f"   Version: {metadata.format_version}")
            print(f"   Created by: {metadata.created_by or 'Unknown'}")
    except Exception as e:
        is_parquet = False
        if verbose:
            print(f"❌ VALIDATION: Not a valid Parquet file")
            print(f"   Error: {e}")

        # Check file extension
        if path.suffix.lower() not in ['.parquet', '.pq']:
            print(f"⚠️  Warning: File extension is '{path.suffix}', not .parquet or .pq")

        return None

    # Test 2: Read the file (full or partial)
    try:
        # Method 1: Read just schema (fastest)
        table = pq.read_table(filepath)

        # Get basic info
        num_rows = table.num_rows
        num_columns = table.num_columns
        column_names = table.column_names

        if verbose:
            print(f"📊 BASIC STATS:")
            print(f"   Rows: {num_rows:,}")
            print(f"   Columns: {num_columns}")

        # Test 3: Column information
        if verbose:
            print(f"\n📋 COLUMNS ({num_columns} total):")
            for i, col in enumerate(column_names, 1):
                col_type = str(table.column(col).type)
                print(f"   {i:2d}. {col:<30} ({col_type})")

        # Test 4: Data preview
        if verbose and num_rows > 0:
            print(f"\n👀 DATA PREVIEW (first {min(sample_rows, num_rows)} rows):")

            # Convert to pandas for easy display
            df_sample = table.slice(0, sample_rows).to_pandas()
            with pd.option_context('display.max_columns', None,
                                 'display.width', None,
                                 'display.max_colwidth', 30):
                print(df_sample.to_string(index=False))

        # Test 5: Additional metadata
        schema = table.schema
        if verbose:
            print(f"\n📝 SCHEMA METADATA:")
            print(f"   Pandas metadata: {'Yes' if schema.pandas_metadata else 'No'}")

            # Check for field metadata
            for field in schema:
                if field.metadata:
                    print(f"   Field '{field.name}' has custom metadata")

        # Test 6: Partition information (if applicable)
        try:
            dataset = pq.ParquetDataset(filepath)
            if dataset.partitioning:
                if verbose:
                    print(f"\n🔗 PARTITIONED DATASET:")
                    print(f"   Partition schema: {dataset.partitioning}")
        except:
            pass  # Not a partitioned dataset

        # Compile results
        results = {
            'filepath': str(path.absolute()),
            'filename': path.name,
            'is_valid_parquet': True,
            'num_rows': num_rows,
            'num_columns': num_columns,
            'column_names': column_names,
            'column_types': {col: str(table.column(col).type) for col in column_names},
            'file_size_bytes': path.stat().st_size,
            'file_size_mb': path.stat().st_size / (1024 * 1024),
            'metadata': {
                'format_version': metadata.format_version,
                'num_row_groups': metadata.num_row_groups,
                'created_by': metadata.created_by,
            },
            'has_pandas_metadata': bool(schema.pandas_metadata),
        }

        if verbose:
            print(f"\n✅ All tests passed!")
            print(f"{'='*60}")

        return results

    except Exception as e:
        if verbose:
            print(f"\n❌ ERROR reading Parquet file:")
            print(f"   {type(e).__name__}: {e}")
        return None


### Defining Filepaths

In [48]:
!pwd

/Users/Daniel/Desktop/open-source-marginal-emissions.nosync/notebooks


In [49]:
current_dir = os.getcwd()
base_dir = os.path.dirname(current_dir)

print_directory_tree(base_dir, show_files=False)

📁 open-source-marginal-emissions.nosync
│   └── 📁 information
│   └── 📁 logs
│   │   └── 📁 weather_data_retrieval
│   │   └── 📁 grid_data_retrieval
│   └── 📁 configs
│   │   └── 📁 pipelines
│   │   └── 📁 weather
│   │   └── 📁 grid
│   └── 📁 packages
│   │   └── 📁 osme_common
│   │   │   └── 📁 tests
│   │   │   └── 📁 docs
│   │   │   └── 📁 src
│   │   │   │   └── 📁 osme_common
│   │   │   │   │   └── 📁 __pycache__
│   │   └── 📁 weather_data_retrieval
│   │   │   └── 📁 tests
│   │   │   └── 📁 docs
│   │   │   └── 📁 src
│   │   │   │   └── 📁 weather_data_retrieval
│   │   │   │   │   └── 📁 io
│   │   │   │   │   │   └── 📁 __pycache__
│   │   │   │   │   └── 📁 utils
│   │   │   │   │   │   └── 📁 __pycache__
│   │   │   │   │   └── 📁 __pycache__
│   │   │   │   │   └── 📁 sources
│   │   │   │   │   │   └── 📁 __pycache__
│   │   └── 📁 tests
│   │   └── 📁 grid_data_retrieval
│   │   │   └── 📁 tests
│   │   │   └── 📁 docs
│   │   │   └── 📁 src
│   │   │   │   └── 📁 grid_data_retrieval
│   │   

In [50]:
grid_data_dir = base_dir + "/data/grid_data/raw/"
grid_monthly_data_dir = grid_data_dir + "monthly/"
show_dir_contents(grid_data_dir)

Contents of current directory:
  -> .DS_Store
  -> carbontracker_grid-data_2018-11_2026-02.parquet
  -> monthly
  -> monthly/carbontracker_grid-data_2022_01.parquet
  -> monthly/carbontracker_grid-data_2022_11.parquet
  -> monthly/carbontracker_grid-data_2023_07.parquet
  -> monthly/carbontracker_grid-data_2022_08.parquet
  -> monthly/carbontracker_grid-data_2021_04.parquet
  -> monthly/carbontracker_grid-data_2018_11.parquet
  -> monthly/carbontracker_grid-data_2024_05.parquet
  -> monthly/carbontracker_grid-data_2019_07.parquet
  -> monthly/carbontracker_grid-data_2020_12.parquet
  -> monthly/carbontracker_grid-data_2020_02.parquet
  -> monthly/carbontracker_grid-data_2025_03.parquet
  -> monthly/carbontracker_grid-data_2022_09.parquet
  -> monthly/carbontracker_grid-data_2021_05.parquet
  -> monthly/carbontracker_grid-data_2024_04.parquet
  -> monthly/carbontracker_grid-data_2020_03.parquet
  -> monthly/carbontracker_grid-data_2019_06.parquet
  -> monthly/carbontracker_grid-data_202

In [51]:
test_filepath_1 = grid_data_dir + "carbontracker_grid-data_2018-11_2026-02.parquet"
test_filepath_2 = grid_monthly_data_dir + "carbontracker_grid-data_2022_01.parquet"

### Testing Files

In [52]:
print(get_mime_type(test_filepath_1))
print(read_magic(test_filepath_1))

application/octet-stream
b'PAR1\x15\x00\x15\xd0'


In [53]:
results = test_parquet_file(test_filepath_1, verbose=True)
if results:
    print("\n📋 Summary:")
    print(f"   File: {results['filename']}")
    print(f"   Rows: {results['num_rows']:,}")
    print(f"   Columns: {results['num_columns']}")
    print(f"   Size: {results['file_size_mb']:.2f} MB")


🧪 TESTING PARQUET FILE: carbontracker_grid-data_2018-11_2026-02.parquet
📁 Location: /Users/Daniel/Desktop/open-source-marginal-emissions.nosync/data/grid_data/raw
📊 Size: 29,943,147 bytes (28.56 MB)
✅ VALIDATION: Confirmed as Parquet file
   Version: 1.0
   Created by: Polars
📊 BASIC STATS:
   Rows: 738,533
   Columns: 12

📋 COLUMNS (12 total):
    1. timestamp                      (timestamp[us])
    2. thermal_generation             (double)
    3. gas_generation                 (double)
    4. g_co2_per_kwh                  (double)
    5. hydro_generation               (double)
    6. nuclear_generation             (double)
    7. renewable_generation           (double)
    8. tons_co2                       (double)
    9. total_generation               (double)
   10. tons_co2_per_mwh               (double)
   11. demand_met                     (double)
   12. net_demand                     (double)

👀 DATA PREVIEW (first 5 rows):
          timestamp  thermal_generation  gas_gene

#### Testing File 1

In [54]:
file1 = pl.read_parquet(test_filepath_1)

In [74]:
print(f"Schema of file [{Path(test_filepath_1).stem}]:")
file1.schema

Schema of file [carbontracker_grid-data_2018-11_2026-02]:


Schema([('timestamp', Datetime(time_unit='us', time_zone=None)),
        ('thermal_generation', Float64),
        ('gas_generation', Float64),
        ('g_co2_per_kwh', Float64),
        ('hydro_generation', Float64),
        ('nuclear_generation', Float64),
        ('renewable_generation', Float64),
        ('tons_co2', Float64),
        ('total_generation', Float64),
        ('tons_co2_per_mwh', Float64),
        ('demand_met', Float64),
        ('net_demand', Float64)])

In [75]:
print(f"Descriptive statistics for file [{Path(test_filepath_1).stem}]:")
file1.describe()

Descriptive statistics for file [carbontracker_grid-data_2018-11_2026-02]:


statistic,timestamp,thermal_generation,gas_generation,g_co2_per_kwh,hydro_generation,nuclear_generation,renewable_generation,tons_co2,total_generation,tons_co2_per_mwh,demand_met,net_demand
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""738533""",738533.0,738533.0,738533.0,738533.0,738533.0,738533.0,738533.0,738533.0,738533.0,738533.0,738533.0
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""","""2022-06-12 00:23:24.297573""",125133.185798,4000.167868,732.741137,18067.987236,4997.646685,17597.130467,10310.820982,169796.118055,0.732741,168693.335003,152198.987588
"""std""",null,20785.13317,1627.6042,78.13044,8755.050918,796.631973,15139.70143,1676.35866,27167.00759,0.07813,26902.07532,22276.736984
"""min""","""2018-11-21 10:00:00""",57543.0,0.0,432.234534,0.0,2275.0,478.5,4715.9916,88658.0,0.432235,87972.0,76808.0
"""25%""","""2020-08-26 15:50:00""",109593.0,2576.5,680.889162,10222.0,4393.5,6525.0,9061.1836,150455.0,0.680889,149466.0,136617.0
"""50%""","""2022-06-10 12:30:00""",124560.0,3803.0,736.139959,17335.5,4984.5,12315.0,10256.86015,168005.5,0.73614,166861.5,150932.0
"""75%""","""2024-03-20 19:35:00""",140915.0,5193.0,795.398806,25056.0,5466.0,23236.0,11558.81665,191169.0,0.795399,189693.0,167306.0
"""max""","""2026-02-05 23:50:00""",179602.0,14591.0,899.116153,45871.5,7085.0,82510.0,14812.6736,258643.0,0.899116,256491.0,225340.0


In [76]:
print(f"First 5 rows of file [{Path(test_filepath_1).stem}]")
file1.head(5)

First 5 rows of file [carbontracker_grid-data_2018-11_2026-02]


timestamp,thermal_generation,gas_generation,g_co2_per_kwh,hydro_generation,nuclear_generation,renewable_generation,tons_co2,total_generation,tons_co2_per_mwh,demand_met,net_demand
datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2018-11-21 10:00:00,121874.0,5468.0,791.409319,9377.0,3829.0,12615.0,10101.2188,153163.0,0.791409,150022.0,140548.0
2018-11-21 10:05:00,121874.0,5468.0,791.409319,9377.0,3829.0,12615.0,10101.2188,153163.0,0.791409,150022.0,140548.0
2018-11-21 10:10:00,121943.0,5456.0,790.636082,9279.0,3831.0,12882.0,10106.3716,153391.0,0.790636,150320.0,140509.0
2018-11-21 10:15:00,122325.5,5441.5,789.346533,9289.0,3828.5,13221.0,10136.88685,154105.5,0.789347,150548.5,140884.5
2018-11-21 10:20:00,122426.0,5439.0,788.917546,9244.0,3829.0,13374.0,10144.9537,154312.0,0.788918,150714.0,140938.0


In [77]:
print(f"Last 5 rows of file [{Path(test_filepath_1).stem}]")
file1.tail(5)

Last 5 rows of file [carbontracker_grid-data_2018-11_2026-02]


timestamp,thermal_generation,gas_generation,g_co2_per_kwh,hydro_generation,nuclear_generation,renewable_generation,tons_co2,total_generation,tons_co2_per_mwh,demand_met,net_demand
datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2026-02-05 23:30:00,143896.5,2204.0,832.787594,7241.0,6696.5,9518.5,11767.0458,169556.5,0.832788,167699.5,160038.0
2026-02-05 23:35:00,143446.0,2199.0,832.968415,7098.0,6708.0,9539.0,11730.2777,168990.0,0.832968,167149.0,159451.0
2026-02-05 23:40:00,143119.5,2195.5,832.596436,7103.5,6697.0,9566.0,11703.63465,168681.5,0.832596,166995.0,159115.5
2026-02-05 23:45:00,142477.0,2190.5,832.957891,6971.0,6692.5,9523.0,11651.27615,167854.0,0.832958,166112.0,158331.0
2026-02-05 23:50:00,141565.0,2177.0,833.706932,6775.0,6682.0,9431.0,11576.7155,166630.0,0.833707,164906.0,157199.0


#### Testing File 2

In [68]:
file2 = pl.read_parquet(test_filepath_2)

In [69]:
print(f"Schema of file [{Path(test_filepath_2).stem}]:")
file2.schema

Schema of file [carbontracker_grid-data_2022_01]:


Schema([('timestamp', Datetime(time_unit='us', time_zone=None)),
        ('thermal_generation', Float64),
        ('gas_generation', Float64),
        ('g_co2_per_kwh', Float64),
        ('hydro_generation', Float64),
        ('nuclear_generation', Float64),
        ('renewable_generation', Float64),
        ('tons_co2', Float64),
        ('total_generation', Float64),
        ('tons_co2_per_mwh', Float64),
        ('demand_met', Float64),
        ('net_demand', Float64)])

In [70]:
print(f"Descriptive statistics for file [{Path(test_filepath_2).stem}]:")
file2.describe()

Descriptive statistics for file [carbontracker_grid-data_2022_01]:


statistic,timestamp,thermal_generation,gas_generation,g_co2_per_kwh,hydro_generation,nuclear_generation,renewable_generation,tons_co2,total_generation,tons_co2_per_mwh,demand_met,net_demand
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""8705""",8705.0,8705.0,8705.0,8705.0,8705.0,8705.0,8705.0,8705.0,8705.0,8705.0,8705.0
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""","""2022-01-16 11:59:08.443423""",121708.818476,2867.121348,793.678103,10098.775972,5053.499847,12156.71143,9990.273111,151884.927073,0.793678,150860.391097,139728.215642
"""std""",null,12665.480593,212.862449,44.080517,4743.314573,276.911407,9917.320694,1033.262306,20568.349883,0.044081,20975.216717,16492.638418
"""min""","""2022-01-01 00:00:00""",93377.0,2026.5,698.809457,3924.0,4191.5,1234.0,7685.8249,113086.0,0.698809,111851.0,105562.5
"""25%""","""2022-01-08 16:10:00""",111152.0,2733.0,759.265681,6480.0,5042.5,4524.0,9129.8467,131857.0,0.759266,130427.0,125732.5
"""50%""","""2022-01-16 11:40:00""",123845.0,2834.0,801.377308,8426.5,5068.0,7368.5,10163.03675,155866.0,0.801377,154900.5,140548.0
"""75%""","""2022-01-24 10:00:00""",132338.5,2988.0,831.045785,13018.0,5094.5,20216.0,10855.5083,168792.0,0.831046,167857.0,153398.0
"""max""","""2022-01-31 23:55:00""",143210.5,3676.0,868.611042,25441.0,5794.0,38497.0,11746.9301,192730.5,0.868611,191859.5,174792.0


In [71]:
print(f"First 5 rows of file [{Path(test_filepath_2).stem}]")
file2.head(5)

First 5 rows of file [carbontracker_grid-data_2022_01]


timestamp,thermal_generation,gas_generation,g_co2_per_kwh,hydro_generation,nuclear_generation,renewable_generation,tons_co2,total_generation,tons_co2_per_mwh,demand_met,net_demand
datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2022-01-01 00:00:00,102621.5,2990.0,796.45305,6214.0,5657.0,9756.5,8444.9908,127239.0,0.796453,125349.5,117482.5
2022-01-01 00:05:00,102004.0,3016.0,797.412641,5854.0,5671.0,9801.0,8395.8248,126346.0,0.797413,124359.0,116545.0
2022-01-01 00:10:00,102057.5,3029.0,799.367835,5710.0,5672.0,9641.0,8400.6565,126109.5,0.799368,124152.5,116468.5
2022-01-01 00:15:00,102272.0,3040.5,800.377615,5474.0,5674.0,9757.5,8418.50515,126218.0,0.800378,124222.5,116460.5
2022-01-01 00:20:00,102245.0,3009.0,801.561966,5349.0,5671.0,9707.0,8415.1315,125981.0,0.801562,123929.0,116274.0


In [72]:
print(f"Last 5 rows of file [{Path(test_filepath_2).stem}]")
file2.tail(5)

Last 5 rows of file [carbontracker_grid-data_2022_01]


timestamp,thermal_generation,gas_generation,g_co2_per_kwh,hydro_generation,nuclear_generation,renewable_generation,tons_co2,total_generation,tons_co2_per_mwh,demand_met,net_demand
datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2022-01-31 23:35:00,120589.0,2988.0,845.467504,8224.0,5038.0,3730.0,9903.8768,140569.0,0.845468,138427.0,136839.0
2022-01-31 23:40:00,120329.5,2997.5,845.114957,8226.5,5040.5,3739.5,9883.16165,140333.5,0.845115,138216.0,136594.0
2022-01-31 23:45:00,119819.0,3022.0,845.075349,8093.0,5041.0,3789.5,9842.6278,139764.5,0.845075,137713.5,135975.0
2022-01-31 23:50:00,119258.0,3007.0,844.545107,8065.0,5033.0,3834.0,9796.5121,139197.0,0.844545,137220.0,135363.0
2022-01-31 23:55:00,118799.5,3000.5,844.015797,8053.5,5036.0,3862.0,9759.03815,138751.5,0.844016,136713.5,134889.5
